### Descriptors

**Дескрипторы** Python — это просто объекты, реализующие **протокол дескрипторов**.

Протокол состоит из следующих специальных методов — не все из них обязательны.

- `__get__`: используется для извлечения значения свойства
- `__set__`: используется для хранения значения свойства (мы увидим, где это можно сделать немного позже)
- `__del__`: удаляет свойство из экземпляра
- `__set_name__`: новое в Python 3.6, мы можем использовать это для захвата имени свойства, как оно определяется в классе-владельце (классе, где определяется свойство).

Нам нужно различать два типа дескрипторов:
- **non-data descriptors** (дескрипторы, не являющиеся данными): это дескрипторы, которые реализуют только `__get__` (и, опционально, `__set_name__`)
- **data descriptors** (дескрипторы данных): они реализуют метод `__set__` и, как правило, также метод `__get__`.

Давайте создадим простой дескриптор, не являющийся данными:

In [1]:
from datetime import datetime

class TimeUTC:
    def __get__(self, instance, owner_class):
        return datetime.utcnow().isoformat()

Таким образом, `TimeUTC` — это класс, реализующий только метод `__get__`, и поэтому считается `non-data descriptor`.

Теперь мы можем использовать его для создания свойств в других классах:

In [2]:
class Logger:
    current_time = TimeUTC()

Обратите внимание, что `current_time` — это атрибут класса:

In [3]:
Logger.__dict__

mappingproxy({'__module__': '__main__',
              'current_time': <__main__.TimeUTC at 0x7fdcd84bbd68>,
              '__dict__': <attribute '__dict__' of 'Logger' objects>,
              '__weakref__': <attribute '__weakref__' of 'Logger' objects>,
              '__doc__': None})

Мы можем получить доступ к этому атрибуту из экземпляра класса `Logger`:

In [4]:
l = Logger()

In [5]:
l.current_time

'2019-07-13T20:47:06.391770'

Мы также можем получить к нему доступ из самого класса, и на данный момент он ведет себя так же (мы вернемся к этому позже):

In [6]:
Logger.current_time

'2019-07-13T20:47:06.405059'

Давайте рассмотрим другой пример.

Предположим, мы хотим создать класс, который позволяет нам выбирать случайную масть и случайную карту этой масти из колоды карт (с заменой, т. е. одну и ту же карту можно выбрать более одного раза).

Мы могли бы подойти к этому вопросу следующим образом:

In [7]:
from random import choice, seed

class Deck:
    @property
    def suit(self):
        return choice(('Spade', 'Heart', 'Diamond', 'Club'))

    @property
    def card(self):
        return choice(tuple('23456789JQKA') + ('10',))

In [8]:
d = Deck()

In [9]:
seed(0)

for _ in range(10):
    print(d.card, d.suit)

8 Club
2 Diamond
J Club
8 Diamond
9 Diamond
Q Heart
J Heart
6 Heart
10 Spade
Q Diamond


Это было довольно просто, но, как вы видите, оба свойства по сути делали одно и то же — они выбирали случайный выбор из некоторого iterable.

Давайте перепишем это с использованием пользовательского дескриптора:

In [10]:
class Choice:
    def __init__(self, *choices):
        self.choices = choices

    def __get__(self, instance, owner_class):
        return choice(self.choices)

И теперь мы можем переписать наш класс `Deck` следующим образом:

In [11]:
class Deck:
    suit = Choice('Spade', 'Heart', 'Diamond', 'Club')
    card = Choice(*'23456789JQKA', '10')

In [12]:
seed(0)

d = Deck()

for _ in range(10):
    print(d.card, d.suit)

8 Club
2 Diamond
J Club
8 Diamond
9 Diamond
Q Heart
J Heart
6 Heart
10 Spade
Q Diamond


Конечно, мы не ограничиваемся только карточками, мы могли бы использовать это и в других классах:

In [13]:
class Dice:
    die_1 = Choice(1,2,3,4,5,6)
    die_2 = Choice(1,2,3,4,5,6)
    die_3 = Choice(1,2,3,4,5,6)

In [14]:
seed(0)

dice = Dice()
for _ in range(10):
    print(dice.die_1, dice.die_2, dice.die_3)

4 4 1
3 5 4
4 3 4
3 5 2
5 2 3
2 1 5
3 5 6
5 2 3
1 6 1
6 3 4
